### Импорт библиотек

In [ ]:
import numpy as np
import pyarrow
import pandas as pd
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import roc_auc_score

### Загрузка данных

In [ ]:
train = pd.read_parquet('data/train_main_features.parquet')
train_ext = pd.read_parquet('data/train_extra_features.parquet')
test = pd.read_parquet('data/test_main_features.parquet')
test_ext = pd.read_parquet('data/test_extra_features.parquet')
target = pd.read_parquet('data/train_target.parquet')
sample_submit = pd.read_parquet('data/sample_submit.parquet')

print('Тренировочные данные:', train.shape)
print('Тренировочные данные extra:', train_ext.shape)
print('Тестовые данные:', test.shape)
print('Тестовые данные extra:', test_ext.shape)

Тренировочные данные: (750000, 200)
Тренировочные данные: (750000, 2242)
Тестовые данные: (250000, 200)
Тестовые данные: (250000, 2242)


### Объединение данных

In [ ]:
train_full = train.copy()
cols_to_add = [col for col in train_ext.columns if col not in train.columns]
train_full[cols_to_add] = train_ext[cols_to_add]

test_full = test.copy()
cols_to_add = [col for col in test_ext.columns if col not in test.columns]
test_full[cols_to_add] = test_ext[cols_to_add]


C:\Users\PC-Fle3n\AppData\Local\Temp\ipykernel_18192\242601614.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_full[cols_to_add] = train_ext[cols_to_add]
C:\Users\PC-Fle3n\AppData\Local\Temp\ipykernel_18192\242601614.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_full[cols_to_add] = train_ext[cols_to_add]
C:\Users\PC-Fle3n\AppData\Local\Temp\ipykernel_18192\242601614.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

In [4]:
print('Полные тренировочные данные:', train_full.shape)
print('Полные тестовые данные:', test_full.shape)

Полные тренировочные данные: (750000, 2441)
Полные тестовые данные: (750000, 2441)


In [6]:
train_full.isnull().sum().sum()

np.int64(1027782741)

In [ ]:
cat_feature_names = [
    col_name for col_name in train_full.columns 
    if col_name.startswith("cat_feature")
]

train_full[cat_feature_names] = train_full[cat_feature_names].astype(str)
test_full[cat_feature_names] = test_full[cat_feature_names].astype(str)

### Обучение модели (13 минут обучения на RTX 4080)

In [ ]:
train_pool = Pool(data = train_full.drop("customer_id", axis=1), 
                  label = target.drop("customer_id", axis=1), 
                  cat_features = cat_feature_names)

model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function="MultiLogloss",
    random_seed=42,
    verbose=100,
    task_type="GPU",
)
model.fit(train_pool)

0:	learn: 0.4839984	total: 1.19s	remaining: 19m 45s
100:	learn: 0.0855812	total: 1m 25s	remaining: 12m 36s
200:	learn: 0.0832911	total: 2m 46s	remaining: 11m
300:	learn: 0.0818387	total: 4m 8s	remaining: 9m 36s
400:	learn: 0.0808172	total: 5m 28s	remaining: 8m 10s
500:	learn: 0.0801149	total: 6m 46s	remaining: 6m 45s
600:	learn: 0.0796137	total: 8m 2s	remaining: 5m 20s
700:	learn: 0.0791833	total: 9m 16s	remaining: 3m 57s
800:	learn: 0.0787906	total: 10m 30s	remaining: 2m 36s
900:	learn: 0.0784651	total: 11m 44s	remaining: 1m 17s
999:	learn: 0.0781780	total: 12m 57s	remaining: 0us


### Предсказание и генерация submission

In [ ]:
test_pool = Pool(data = test_full.drop("customer_id", axis=1), 
                 cat_features = cat_feature_names)
test_predict = model.predict(test_pool, prediction_type = "RawFormulaVal")
result_df = sample_submit.copy()
result_df.iloc[:, 1:] = test_predict
result_df['customer_id'] = result_df['customer_id'].astype('int32')
current_type = result_df['customer_id'].dtype
result_df.to_parquet('submission.parquet', index=False)

Текущий тип customer_id: int32
Файл сохранен.
